In [ ]:
#### ANOMALY DETECTION WITH DIFFUSION ####

In [ ]:
# ============================================================
# IMPORTS
# ============================================================
import os
import time
import psutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.fft import fft
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, average_precision_score
)

# ============================================================
# LOGGER
# ============================================================
def log(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# ============================================================
# CONFIGURATION
# ============================================================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
WINDOW_SIZE = 24
BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-4

ANOMALY_IMPUTATION_TYPE = "AnomalyDiffusion"
BASE_INPUT = f"/content/drive/MyDrive/Masters_IndependentStudy/{ANOMALY_IMPUTATION_TYPE}_labelled"
BASE_OUTPUT = f"/content/drive/MyDrive/Masters_IndependentStudy/{ANOMALY_IMPUTATION_TYPE}_prediction"
STATS_FILE = f"/content/drive/MyDrive/Masters_IndependentStudy/{ANOMALY_IMPUTATION_TYPE}_Statistics.csv"

RESIDENCES = [
    "AMPds2_House01",
    "GREEND_House00", "GREEND_House01", "GREEND_House03",
    "UKDALE_House01", "UKDALE_House02", "UKDALE_House05",
    "REFIT_House01", "REFIT_House02", "REFIT_House03", "REFIT_House05",
    "REFIT_House07", "REFIT_House09", "REFIT_House15"
]

ANOMALY_TYPES = [
    "stepchange", "multistepchange", "mirror",
    "repeating", "stuckmax", "stuckmin", "powercycling"
]

THRESHOLDS = [70] #[70, 75, 80, 85, 90]  # Percentiles to loop through

# ============================================================
# DATASET AND WINDOWS
# ============================================================
class PowerDataset(Dataset):
    def __init__(self, series):
        self.series = series
    def __len__(self):
        return len(self.series)
    def __getitem__(self, idx):
        return torch.tensor(self.series[idx], dtype=torch.float32).unsqueeze(0)

def create_windows(signal, window=WINDOW_SIZE):
    return np.array([signal[i:i + window] for i in range(0, len(signal) - window + 1, window)])

# ============================================================
# SIMPLE UNET1D
# ============================================================
class UNet1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.dec = nn.Conv1d(16, 1, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
    def forward(self, x):
        e = self.relu(self.enc(x))
        d = self.dec(e)
        return d + x  # skip connection

# ============================================================
# DDPM SCHEDULER
# ============================================================
class DDPM:
    def __init__(self, T=1000):
        self.T = T
        self.beta = torch.linspace(1e-4, 0.02, T).to(DEVICE)
        self.alpha = 1.0 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
    def add_noise(self, x0, t):
        eps = torch.randn_like(x0)
        a_bar = self.alpha_bar[t].view(-1, 1, 1)

        #### FUNDAMENTAL EQUATION - FORWARD PASS ####
        xt = torch.sqrt(a_bar) * x0 + torch.sqrt(1 - a_bar) * eps
        return xt, eps

# ============================================================
# TRAINING
# ============================================================
def train_model(train_loader):
    log("Initializing UNet1D diffusion model")
    model = UNet1D().to(DEVICE)
    ddpm = DDPM()
    opt = torch.optim.Adam(model.parameters(), lr=LR)
    loss_fn = nn.MSELoss()

    start_time = time.time()
    model.train()
    for epoch in range(EPOCHS):
        log(f"Epoch {epoch + 1}/{EPOCHS} started")
        for x in train_loader:
            x = x.to(DEVICE)
            t = torch.randint(0, ddpm.T, (x.size(0),)).to(DEVICE)
            xt, eps = ddpm.add_noise(x, t)
            eps_hat = model(xt)
            loss = loss_fn(eps_hat, eps)
            opt.zero_grad()
            loss.backward()
            opt.step()
        log(f"Epoch {epoch + 1} completed")
    elapsed = time.time() - start_time
    log(f"Training finished in {elapsed:.2f} seconds")
    return model, elapsed

# ============================================================
# ANOMALY SCORE COMPUTATION
# ============================================================
def compute_scores(x, x_hat):
    mae_t = np.mean(np.abs(x - x_hat), axis=1)
    rx = np.log1p(np.abs(fft(x)))
    rx_hat = np.log1p(np.abs(fft(x_hat)))
    mae_f = np.mean(np.abs(rx - rx_hat), axis=1)
    def z_robust(v):
        med = np.median(v)
        mad = np.median(np.abs(v - med)) + 1e-6
        return (v - med) / mad

    #### FUNDAMENTAL EQUATION - ANOMALY SCORE ####
    score = 0.5 * (z_robust(mae_t) + z_robust(mae_f))
    return score, mae_t, mae_f

# ============================================================
# METRICS HELPERS
# ============================================================
def cohen_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    pooled = np.sqrt(((n1-1)*s1 + (n2-1)*s2) / (n1 + n2 -2))
    if pooled == 0: return 0.0
    return (np.mean(x1) - np.mean(x2)) / pooled

def separation_delta(x1, x2):
    return np.mean(x1) - np.mean(x2)

# ============================================================
# MAIN PIPELINE - TRAINING, COMPUTE SCORES, AND INFERENCE
# ============================================================
log("Starting diffusion anomaly detection pipeline")
stats = []

for residence in RESIDENCES:
    log(f"Processing residence: {residence}")

    # ------------------ TRAINING ---------------------
    train_path = f"{BASE_INPUT}/{residence}_Fridge_stepchange_{ANOMALY_IMPUTATION_TYPE}_labelled.csv"
    log(f"Loading training data: {train_path}")
    df_train = pd.read_csv(train_path)
    signal_train = df_train["active_power"].values
    windows_train = create_windows(signal_train)
    split = int(0.8 * len(windows_train))
    train_windows = windows_train[:split]
    train_ds = PowerDataset(train_windows)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

    train_peak_mb = psutil.Process().memory_info().rss / (1024 ** 2)
    model, train_time = train_model(train_loader)

    # ------------------ COMPUTE SCORES ---------------------
    model.eval()
    with torch.no_grad():
        recon_train = model(torch.tensor(train_windows).float().unsqueeze(1).to(DEVICE)).cpu().numpy().squeeze()
    scores_train, _, _ = compute_scores(train_windows, recon_train)
    varthreshold = np.var(scores_train)

    # ------------------ INFERENCE ---------------------
    for anomaly_type in ANOMALY_TYPES:
        log(f"Inference for anomaly type: {anomaly_type}")
        path = f"{BASE_INPUT}/{residence}_Fridge_{anomaly_type}_{ANOMALY_IMPUTATION_TYPE}_labelled.csv"
        df_infer = pd.read_csv(path)

        # Normalize ground truth
        df_infer["anomaly_groundtruth"] = df_infer["anomaly_groundtruth"].astype(str).str.strip().str.lower()

        # Actual counts
        ActualNormal = np.sum(df_infer["anomaly_groundtruth"] == "normal")
        ActualAnomaly = np.sum(df_infer["anomaly_groundtruth"] == "anomaly")

        # Create windows
        windows_infer = create_windows(df_infer["active_power"].values)
        start_inf = time.time()
        with torch.no_grad():
            recon_infer = model(torch.tensor(windows_infer).float().unsqueeze(1).to(DEVICE)).cpu().numpy().squeeze()

        scores_windows, _, _ = compute_scores(windows_infer, recon_infer)

        # Propagate scores to rows
        scores_per_row = np.zeros(len(df_infer))
        for i, score in enumerate(scores_windows):
            start_idx = i * WINDOW_SIZE
            end_idx = start_idx + WINDOW_SIZE
            scores_per_row[start_idx:end_idx] = score
        scores_per_row = scores_per_row[:len(df_infer)]

        # ------------------ LOOP THROUGH THRESHOLDS ---------------------
        # For testing purposes we loop through many thresholds
        # - During inference, only one threshold is mentioned in the variable "THRESHOLDS"
        for THRESHOLD_PCT in THRESHOLDS:
            threshold = np.percentile(scores_train, THRESHOLD_PCT)
            preds = np.where(scores_per_row > threshold, "anomaly", "normal")
            df_out = df_infer.copy()
            df_out["anomaly_prediction"] = preds
            os.makedirs(BASE_OUTPUT, exist_ok=True)
            out_path = f"{BASE_OUTPUT}/{residence}_Fridge_{anomaly_type}_{ANOMALY_IMPUTATION_TYPE}_prediction_{THRESHOLD_PCT}.csv"
            df_out.to_csv(out_path, index=False)
            log(f"Predictions saved: {out_path}")

            # Metrics
            y_true = np.where(df_out["anomaly_groundtruth"] == "anomaly", 1, 0)
            y_pred = (preds == "anomaly").astype(int)
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
            normal_pct = 100 * np.sum(y_pred == 0) / len(y_pred)
            anomaly_pct = 100 * np.sum(y_pred == 1) / len(y_pred)
            inf_peak_mb = psutil.Process().memory_info().rss / (1024 ** 2)

            # ROC AUC and PR AUC on thresholded predictions
            if len(np.unique(y_pred)) < 2:
                roc_auc = 0.0
                pr_auc = 0.0
            else:
                roc_auc = roc_auc_score(y_true, y_pred)
                pr_auc = average_precision_score(y_true, y_pred)

            # Cohen's d and separation delta on thresholded predictions
            normal_scores = y_pred[y_true==0]
            anomaly_scores = y_pred[y_true==1]
            if len(normal_scores) > 0 and len(anomaly_scores) > 0:
                d_cohen = cohen_d(normal_scores, anomaly_scores)
                separation = separation_delta(normal_scores, anomaly_scores)
            else:
                d_cohen = 0.0
                separation = 0.0

            stats.append({
                "residence": residence,
                "appliance": "Fridge",
                "anomaly type": anomaly_type,
                "thresholdpct": THRESHOLD_PCT,
                "thresholdvalue": threshold,
                "varthreshold": varthreshold,
                "windows": len(windows_infer),
                "trainingtimesec": train_time,
                "inferencetimesec": time.time() - start_inf,
                "trainpeakmb": train_peak_mb,
                "inferencepeakmb": inf_peak_mb,
                "accuracy": accuracy_score(y_true, y_pred),
                "precision": precision_score(y_true, y_pred, zero_division=0),
                "recall": recall_score(y_true, y_pred, zero_division=0),
                "f1_score": f1_score(y_true, y_pred, zero_division=0),
                "normal_%": normal_pct,
                "anomaly_%": anomaly_pct,
                "total": len(y_true),
                "TP": tp, "TN": tn, "FP": fp, "FN": fn,
                "ActualNormal": ActualNormal,
                "ActualAnomaly": ActualAnomaly,
                "roc_auc": roc_auc,
                "pr_auc": pr_auc,
                "cohens_d": d_cohen,
                "separation_delta": separation
            })

# ============================================================
# SAVE FINAL STATISTICS
# ============================================================
log("Saving statistics CSV")
pd.DataFrame(stats).to_csv(STATS_FILE, index=False)
log("✅ Pipeline completed successfully")
